----
# TensorFlow Tutorial
(*이 자료는
[TensorFlow Tutorial](https://www.slideshare.net/nmhkahn/tensorflow-tutorial-71896086 "링크")을 참조하여 만들어짐*)

----
## TensorFlow의 장점
1. Easy numerical computation
2. Automaticderivatives computation
3. Deploy computation to multiple CPUs or GPUs
4. Powerful data visualization toolkit

## Tensor란?
1. vector와 matrix의 확장형
2. N-Dimension array로 볼 수 있음

In Numpy (1)

In [1]:
import numpy as np
a = np.zeros((2,2))
b = np.ones((2,2))
np.sum(b, axis=1)


array([ 2.,  2.])

In [2]:
a.shape

(2, 2)

In [3]:
np.reshape(a, (1,4))

array([[ 0.,  0.,  0.,  0.]])

In TensorFlow (1)

In [4]:
import tensorflow as tf
sess = tf.Session()

a = tf.zeros((2,2))
b = tf.ones((2,2))
sess.run(tf.reduce_sum(b, axis=1))

array([ 2.,  2.], dtype=float32)

In [5]:
a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

In [6]:
sess.run(tf.reshape(a, (1,4)))

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

sess.run()

In [7]:
sess = tf.Session()
a = np.zeros((2,2))
ta = tf.zeros((2,2))

print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [8]:
print(ta) # TnsorFlow는 처음에 값이 없는 Computation graph를 정의함

Tensor("zeros_1:0", shape=(2, 2), dtype=float32)


In [9]:
print(sess.run(ta))

[[ 0.  0.]
 [ 0.  0.]]


Computation Graph

In [10]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b           # 이 과정은 Computation graph를 정의하는 과정

In [11]:
print(sess.run(c))  # 이 과정은 이전에 정의된 Computation graph에 따라 계산

30.0


TensorFlow Session

In [12]:
with tf.Session() as sess:
    print(sess.run(c))
    print(c.eval())    # c.eval()은 sess 지역안에 있어야 함

30.0
30.0


TensorFlow Variables (no initialize)

In [13]:
w = tf.Variable(tf.zeros((2,2)), name="weight")
#with tf.Session() as sess:
    #print(sess.run(w))    # 변수가 초기화 되지 않았으므로 에러!!

Variables need ** *Initialization* **

In [14]:
w = tf.Variable(tf.random_normal([5,2], stddev = 0.1), name="weight")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[-0.1119767  -0.09501576]
 [ 0.08057523  0.18241394]
 [-0.0219014  -0.08727607]
 [-0.0773242  -0.08091877]
 [ 0.10698026  0.05581374]]


Updating Variables

In [15]:
state = tf.Variable(0, name="counter")
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


Fetching Variables

In [16]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3)
mul = tf.multiply(x1, temp)

with tf.Session() as sess:
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

5 5


**TensorFlow Placeholder**  
이전까지의 예제는 모두 수동으로 정의한 Tensors.  
외부데이터를 Tensorflow에 넣기 위해서 placeholder와 feed_dict를 사용함

In [18]:
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

add = tf.add(a,b)
mul = tf.multiply(a,b)

with tf.Session() as sess:
    print(sess.run(add, feed_dict={a:2, b:3}))   # feed_dict : python dictionary
    print(sess.run(mul, feed_dict={a:2, b:3}))

5
6


In [19]:
#using tf.constant
mat1 = tf.constant([[3., 3.]])
mat2 = tf.constant([[2.], [2.]])
product = tf.matmul(mat1, mat2)

with tf.Session() as sess:
    result = sess.run(product)
    print(result)
    

[[ 12.]]


In [24]:
#using placeholder
import numpy as np

mat1 = tf.placeholder(tf.float32, [1, 2])
mat2 = tf.placeholder(tf.float32, [2, 1])
product = tf.matmul(mat1, mat2)

with tf.Session() as sess:
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.], [2.]])
    result = sess.run(product, feed_dict={mat1 : mv1, mat2 : mv2})
    print(result)



[[ 12.]]


### Example - MNIST with MLP

In [25]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [26]:
learning_rate = 0.001
max_steps = 15000
batch_size =128

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [31]:
def MLP(inputs):
    w_1 = tf.Variable(tf.random_normal([784, 256]))
    b_1 = tf.Variable(tf.zeros([256]))
    
    w_2 = tf.Variable(tf.random_normal([256, 256]))
    b_2 = tf.Variable(tf.zeros([256]))
    
    w_out = tf.Variable(tf.random_normal([256, 10]))
    b_out = tf.Variable(tf.zeros([10]))
    
    h_1 = tf.add(tf.matmul(inputs, w_1), b_1)
    h_1 = tf.nn.relu(h_1)
    
    h_2 = tf.add(tf.matmul(h_1, w_2), b_2)
    h_2 = tf.nn.relu(h_2)
    
    out = tf.add(tf.matmul(h_2, w_out), b_out)
    
    return out

net = MLP(x)


In [36]:

#define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=y))         # 수정
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)


# initializing the variables
init_op = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init_op)

#train model
for step in range(max_steps):
    batch_X, batch_y = mnist.train.next_batch(batch_size)
    _, loss = sess.run([opt, loss_op], feed_dict={x:batch_X, y:batch_y})
    
    if (step +1) % 1000 == 0:
        print("[{}/{}] loss : {:.3f}".format(step+1, max_steps, loss))
print("Optimization Finished!")

#test model
correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))

# calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Train accuracy :{:.3f}".format(sess.run(accuracy, feed_dict={x:mnist.train.images, y:mnist.train.labels})))
print("Test accuracy :{:.3f}".format(sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})))

[1000/15000] loss : 58.110
[2000/15000] loss : 18.055
[3000/15000] loss : 6.305
[4000/15000] loss : 3.923
[5000/15000] loss : 1.069
[6000/15000] loss : 0.387
[7000/15000] loss : 0.063
[8000/15000] loss : 0.019
[9000/15000] loss : 0.150
[10000/15000] loss : 0.799
[11000/15000] loss : 0.142
[12000/15000] loss : 0.000
[13000/15000] loss : 0.158
[14000/15000] loss : 0.000
[15000/15000] loss : 0.343
Optimization Finished!
Train accuracy :0.996
Test accuracy :0.954


Variable Scope - variable_scope()

In [41]:
var1 = tf.Variable([1], name="var") 
with tf.variable_scope("foo"): 
    with tf.variable_scope("d"): 
        var2 = tf.Variable([1], name="var") 
        var3 = tf.Variable([1], name="b") 
print("var1: {}".format(var1.name)) 
print("var2: {}".format(var2.name)) 
print("var3: {}".format(var3.name))

var1: var_2:0
var2: foo_2/d/var:0
var3: foo_2/d/b:0


Variable Scope - get_variable()

In [47]:
var1 = tf.Variable([1], name="var") 
with tf.variable_scope("foo"): 
    with tf.variable_scope("bar") as scp: 
        var2 = tf.Variable([1], name="var") 
        scp.reuse_variables() # allow reuse variables
        var3 = tf.Variable([1], name="var") 
print("var1: {}".format(var1.name)) 
print("var2: {}".format(var2.name)) 
print("var3: {}".format(var3.name))    # var3의 이름을 foo/bar/var:0로 하고 싶으나 안됨

var1: var_8:0
var2: foo_8/bar/var:0
var3: foo_8/bar/var_1:0


In [54]:
var1 = tf.get_variable("var", [1])  
with tf.variable_scope("foo"): 
    with tf.variable_scope("bar") as scp: 
        var2 = tf.get_variable("var", [1]) 
        scp.reuse_variables() # allow reuse variables
        var3 = tf.get_variable("var", [1]) 
print("var1: {}".format(var1.name)) 
print("var2: {}".format(var2.name)) 
print("var3: {}".format(var3.name))    # var3의 이름을 foo/bar/var:0로 함

var1: var_9:0
var2: foo/bar/var_2:0
var3: foo/bar/var_2:0


Parameter sharing

In [60]:
with tf.variable_scope("foo2"):
    with tf.variable_scope("bar2") as scp:
        var1 = tf.get_variable("var2", [1])
        scp.reuse_variables()
        var2 = tf.get_variable("var2", [1])
        
    with tf.variable_scope("bar2", reuse=True):
        var3 = tf.get_variable("var2", [1])
    
print("var1: {}".format(var1.name)) 
print("var2: {}".format(var2.name)) 
print("var3: {}".format(var3.name))     


var1: foo2/bar2/var2:0
var2: foo2/bar2/var2:0
var3: foo2/bar2/var2:0


@@ Tip @@  
tf.Variable보다는 tf.get_variable을 사용하자.  
tf.Variable은 파라미터 sharing 불가  
tf.Variable은 initializer과 regularizer을 사용할 수 없음

### Example - MNIST with CNN

In [61]:
from tensorflow.contrib.layers import variance_scaling_initializer 
he_init = variance_scaling_initializer() 
def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None): 
    bottom_shape = bottom.get_shape().as_list()[3] 
    
    with tf.variable_scope(scope or "conv"): 
        W = tf.get_variable("W", 
                            [ksize, ksize, bottom_shape, num_filter], 
                            initializer=he_init) 
        b = tf.get_variable("b", [num_filter], 
                            initializer=tf.constant_initializer(0)) 
        x = tf.nn.conv2d(bottom, W, 
                         strides=[1, stride, stride, 1], 
                         padding=padding) 
        x = tf.nn.relu(tf.nn.bias_add(x, b)) 
        
    return x

In [62]:
def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None): 
    with tf.variable_scope(scope or "maxpool"): 
        pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding=padding) 
        
    return pool

In [63]:
def fc(bottom, num_dims, scope=None): 
    bottom_shape = bottom.get_shape().as_list() 
    if len(bottom_shape) > 2: 
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])]) 
        bottom_shape = bottom.get_shape().as_list() 
        
    with tf.variable_scope(scope or "fc"): 
        W = tf.get_variable("W", [bottom_shape[1], num_dims], initializer=he_init) 
        b = tf.get_variable("b", [num_dims], initializer=tf.constant_initializer(0)) 
        out = tf.nn.bias_add(tf.matmul(bottom, W), b) 
        
    return out 

def fc_relu(bottom, num_dims, scope=None): 
    with tf.variable_scope(scope or "fc"): 
        out = fc(bottom, num_dims, scope="fc") 
        relu = tf.nn.relu(out) 
        
    return relu

In [64]:
keep_prob = tf.placeholder(tf.float32, None) 

def conv_net(x, keep_prob): 
    x = tf.reshape(x, shape=[-1, 28, 28, 1]) 
    
    conv1 = conv(x, 32, 5, scope="conv_1") 
    conv1 = maxpool(conv1, scope="maxpool_1") 
    conv2 = conv(conv1, 64, 5, scope="conv_2") 
    conv2 = maxpool(conv2, scope="maxpool_2") 
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1") 
    fc1 = tf.nn.dropout(fc1, keep_prob) 
    out = fc(fc1, 10, scope="out") 
    
    return out

In [95]:
config = tf.ConfigProto( gpu_options=tf.GPUOptions(allow_growth=True)) 
sess = tf.Session(config=config)

### tf.slim  
#### tf.contrib.slim의 장점  
building, training, evaluation neural networks가 쉬움  
몇몇의 많이 사용되는 모델들이 slim으로 만들어짐  




Layers

In [96]:
#input = ...  #...에 무엇이 들어가야하는지.
#with tf.name_scope('conv1_1') as scope: 
    #kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32, stddev=1e-1), name='weights') 
    #conv = tf.nn.conv2d(input, kernel, [1, 1, 1, 1], padding='SAME') 
    #biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32), trainable=True, name='biases') 
    #bias = tf.nn.bias_add(conv, biases) 
    #conv1 = tf.nn.relu(bias, name=scope)

In [97]:
#input = ... #...에 무엇이 들어가야 하는지..
#net = slim.conv2d(input, 128, [3, 3], padding= 'SAME', scope= 'conv1_1')

In [114]:
import tensorflow.contrib.slim as slim
#import tf.contrib.slim

# 1. simple network generation with slim 
#net = ... #...에 무엇이 들어가야 하는지..
#net = slim.conv2d(net, 256, [3, 3], scope='conv3_1') 
#net = slim.conv2d(net, 256, [3, 3], scope='conv3_2') 
#net = slim.conv2d(net, 256, [3, 3], scope='conv3_3') 
#net = slim.max_pool2d(net, [2, 2], scope='pool3')

# 1. cleaner by repeat operation: 
#net = ... #...에 무엇이 들어가야 하는지..
#net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3') 
#net = slim.max_pool(net, [2, 2], scope= 'pool3')
                                        
# 2. Verbose way: 
#x = slim.fully_connected(x, 32, scope='fc/fc_1') 
#x = slim.fully_connected(x, 64, scope='fc/fc_2') 
#x = slim.fully_connected(x, 128, scope='fc/fc_3')

# 2. Equivalent, TF-Slim way using slim.stack: 
#slim.stack(x, slim.fully_connected, [32, 64, 128], scope='fc')

Initializer와 Regularizer에는 다음과 같은 것들이 있음  
Initializer  
1. tf.truncated_noraml_initialzier (Included in tf)  
2. slim.xavier_initialzier  
3. slim.variance_scaling_initializer  

Regularizer  
1. slim.l1_regularizer
2. slim.l2_regularizer

In [115]:
net = slim.conv2d(inputs, 64, [11, 11], 4, padding='SAME', 
                  weights_initializer=slim.xavier_initializer(), 
                  weights_regularizer=slim.l2_regularizer(0.0005), scope='conv1')


NameError: name 'inputs' is not defined

argscope

In [116]:
he_init = slim.variance_scaling_initializer() 
xavier_init = slim.xavier_initializer() 

with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                    activation_fn=tf.nn.relu, weights_initializer=he_init, 
                    weights_regularizer=slim.l2_regularizer(0.0005)): 
    
    with slim.arg_scope([slim.conv2d], stride=1, padding='SAME'): 
        net = slim.conv2d(inputs, 64, [11, 11], 4, scope='conv1') 
        net = slim.conv2d(net, 256, [5, 5], weights_initializer=xavier_init, scope='conv2') 
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc')

NameError: name 'inputs' is not defined

Losses  
Multiple loss와 regularize loss를 다루기 쉬움

In [117]:
loss1 = slim.losses.softmax_cross_entropy(pred1, label1) 
loss2 = slim.losses.mean_squared_error(pred2, label2) 

total_loss = loss1 + loss2 
slim.losses.get_total_loss(add_regularization_losses=False) 

reg_loss = tf.add_n(slim.losses.get_regularization_losses()) 
total_loss = loss1 + loss2 + reg_loss 

total_loss = slim.losses.get_total_loss()

NameError: name 'pred1' is not defined

Save / Restore

In [118]:
def save(self, ckpt_dir, global_step=None): 
    if self.config.get("saver") is None: 
        self.config["saver"] = tf.train.Saver(max_to_keep=30) 
        
    saver = self.config["saver"] 
    sess = self.config["sess"] 
    
    dirname = os.path.join(ckpt_dir, self.name) 
    
    if not os.path.exists(dirname): 
        os.makedirs(dirname) 
        
    saver.save(sess, dirname, global_step)

In [119]:
def load_latest_checkpoint(self, ckpt_dir, exclude=None): 
    path = tf.train.latest_checkpoint(ckpt_dir) 
    if path is None: 
        raise AssertionError("No ckpt exists in {0}.".format(ckpt_dir)) 
        
    print("Load {} save file".format(path)) 
    self._load(path, exclude) 

def vggdef load_from_path(self, ckpt_path, exclude=None): 
    self._load(ckpt_path, exclude) 
    
def _load(self, ckpt_path, exclude): 
    init_fn = slim.assign_from_checkpoint_fn(ckpt_path, slim.get_variables_to_restore(exclude=exclude), 
                                             ignore_missing_vars=True) 
    init_fn(self.config["sess"])

VGG-16

In [136]:
def vgg_16(inputs, num_classes=1000, is_training=True, dropout_keep_prob=0.5,
          spatial_squeeze=True, scope='vgg_16'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], outputs_collections = end_points_collection):
            
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2,2], scope='pool1')
            
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2,2], scope='pool2')
            
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2,2], scope='pool3')
            
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2,2], scope='pool4')
            
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2,2], scope='pool5')
            
            net = slim.conv2d(net, 4096, [7,7], padding='VALID', scope='fc6')
            net = slim.dropout(net, dropout_keep_prob, is_training = is_training, scope='dropout6')
            
            net = slim.conv2d(net, 4096, [1,1], scope='fc7')
            net = slim.dropout(net, dropout_keep_prob, is_training = is_training, scope='dropout7')
            
            net = slim.conv2d(net, num_classes, [1,1], activation_fn=None, normalizer_fn=None, scope='fc8')
            
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            
            if spatial_squeeze:
                net = tf.squeeze(net, [1,2], name='fc8/squeezed')
                end_points[sc.name +'/fc8'] = net
                
            return net, end_points
    
vgg_16.default_image_size = 224

def vgg_arg_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                       activation_fn=tf.nn.relu,
                       weights_regularizer=slim.l2_regularizer(weight_decay),
                       biases_initializer=tf.zeros_initializer()):
        with slim.arg_scope([slim.conv2d], padding='SAME') as arg_sc:
            return arg_sc

In [137]:
X = tf.placeholder(tf.float32, [None, 224, 224, 3], name="X") 
y = tf.placeholder(tf.int32, [None, 8], name="y") 
is_training = tf.placeholder(tf.bool, name="is_training") 

with slim.arg_scope(vgg_arg_scope()): 
    net, end_pts = vgg_16(X, is_training=is_training, num_classes=1000) 
    
with tf.variable_scope("losses"): 
    cls_loss = slim.losses.softmax_cross_entropy(net, y) 
    reg_loss = tf.add_n(slim.losses.get_regularization_losses()) 
    loss_op = type_loss + reg_loss 
    
with tf.variable_scope("opt"): 
    opt = tf.train.AdamOptimizer(0.001).minimize(loss_op) 
    
self.load_from_path(ckpt_path=VGG_PATH, exclude=["vgg_16/fc8"])

Instructions for updating:
Use tf.losses.softmax_cross_entropy instead.


ValueError: Shapes (?, 1000) and (?, 8) are incompatible

TensorBoard & slim example

In [129]:
# Import MINST data 
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True) 

max_steps = 10000 
batch_size = 128 
lr = 0.001 
keep_prob = 0.5 
weight_decay = 0.0004 
logs_path = "/tmp/tensorflow_logs/example" 

def my_arg_scope(is_training, weight_decay): 
    with slim.arg_scope([slim.conv2d], 
                        activation_fn=tf.nn.relu, 
                        weights_regularizer=slim.l2_regularizer(weight_decay), 
                        weights_initializer=slim.variance_scaling_initializer(), 
                        biases_initializer=tf.zeros_initializer(), stride=1, 
                        padding="SAME"): 
        with slim.arg_scope([slim.dropout], is_training=is_training) as arg_sc: 
            return arg_sc

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [139]:
def my_net(x, keep_prob, outputs_collections="my_net"): 
    x = tf.reshape(x, shape=[-1, 28, 28, 1]) 
    
    with slim.arg_scope([slim.conv2d, slim.max_pool2d], 
                        outputs_collections=outputs_collections): 
        
        net = slim.conv2d(x, 64, [3, 3], scope="conv1") 
        net = slim.max_pool2d(net, [2, 2], scope="pool1") 
        
        net = slim.conv2d(net, 128, [3, 3], scope="conv2") 
        net = slim.max_pool2d(net, [2, 2], scope="pool2") 
        
        net = slim.conv2d(net, 256, [3, 3], scope="conv3") 
        
        # global average pooling 
        net = tf.reduce_mean(net, [1, 2], name="pool3", keep_dims=True) 
        net = slim.dropout(net, keep_prob, scope="dropout3") 
        net = slim.conv2d(net, 1024, [1, 1], scope="fc4") 
        net = slim.dropout(net, keep_prob, scope="dropout4") 
        net = slim.conv2d(net, 10, [1, 1], activation_fn=None, scope="fc5") 
        
    end_points = slim.utils.convert_collection_to_dict(outputs_collections) 
    
    return tf.reshape(net, [-1, 10]), end_points

In [141]:
x = tf.placeholder(tf.float32, [None, 784]) 
y = tf.placeholder(tf.float32, [None, 10]) 
is_training = tf.placeholder(tf.bool) 

with slim.arg_scope(my_arg_scope(is_training, weight_decay)): 
    net, end_pts = my_net(x, keep_prob) 
    pred = slim.softmax(net, scope="prediction") 
    
with tf.variable_scope("losses"): 
    cls_loss = tf.losses.softmax_cross_entropy(net, y) 
    reg_loss = tf.add_n(tf.losses.get_regularization_losses()) 
    loss_op = cls_loss + reg_loss 

with tf.variable_scope("Adam2"):
    opt = tf.train.AdamOptimizer(lr) 
    
    # Op to calculate every variable gradient 
    grads = tf.gradients(loss_op, tf.trainable_variables()) 
    grads = list(zip(grads, tf.trainable_variables())) 
    
    # Op to update all variables according to their gradient 
    apply_grads = opt.apply_gradients(grads_and_vars=grads) 
    
with tf.variable_scope("accuracy"): 
    correct_op = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1)) 
    acc_op = tf.reduce_mean(tf.cast(correct_op, tf.float32))

In [144]:
# Create a summary to monitor loss and accuracy 
summ_loss = tf.summary.scalar("loss", loss_op) 
summ_acc = tf.summary.scalar("accuracy_test", acc_op) 

# Create summaries to visualize weights and grads 
for var in tf.trainable_variables(): 
    tf.summary.histogram(var.name, var, collections=["my_summ"]) 
    
for grad, var in grads: 
    tf.summary.histogram(var.name + "/gradient", grad, collections=["my_summ"]) 
    
summ_wg = tf.summary.merge_all(key="my_summ") 

sess = tf.Session() 
sess.run(tf.global_variables_initializer()) 

summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)

INFO:tensorflow:Summary name weight:0 is illegal; using weight_0 instead.
INFO:tensorflow:Summary name weight_1:0 is illegal; using weight_1_0 instead.
INFO:tensorflow:Summary name counter:0 is illegal; using counter_0 instead.
INFO:tensorflow:Summary name Variable:0 is illegal; using Variable_0 instead.
INFO:tensorflow:Summary name Variable_1:0 is illegal; using Variable_1_0 instead.
INFO:tensorflow:Summary name Variable_2:0 is illegal; using Variable_2_0 instead.
INFO:tensorflow:Summary name Variable_3:0 is illegal; using Variable_3_0 instead.
INFO:tensorflow:Summary name Variable_4:0 is illegal; using Variable_4_0 instead.
INFO:tensorflow:Summary name Variable_5:0 is illegal; using Variable_5_0 instead.
INFO:tensorflow:Summary name Variable_6:0 is illegal; using Variable_6_0 instead.
INFO:tensorflow:Summary name Variable_7:0 is illegal; using Variable_7_0 instead.
INFO:tensorflow:Summary name Variable_8:0 is illegal; using Variable_8_0 instead.
INFO:tensorflow:Summary name Variable_

ValueError: None values not supported.

In [143]:
for step in range(max_steps): 
    batch_X, batch_y = mnist.train.next_batch(batch_size) 
    _, loss, plot_loss, plot_wg = sess.run([apply_grads, loss_op, summ_loss, summ_wg], 
                                           feed_dict={x: batch_X, y: batch_y, is_training: True}) 
    
    summary_writer.add_summary(plot_loss, step) 
    summary_writer.add_summary(plot_wg, step) 
    
    if (step+1) % 100 == 0: 
        plot_acc = sess.run(summ_acc, feed_dict={x: mnist.test.images, y: mnist.test.labels, is_training: False}) 
        summary_writer.add_summary(plot_acc, step) 
        
print("Optimization Finished!") 

test_acc = sess.run(acc_op, feed_dict={x: mnist.test.images, y: mnist.test.labels, is_training: False}) 

print("Test accuracy: {:.3f}".format(test_acc))

NameError: name 'summ_wg' is not defined